Multivariate Cryptography

Introduction to Quantum-Safe Cryptography (IBM Zurich)

Simona Samardjiska

Programming assignment

Implementation of the UOV signature scheme

the implementation matches the general design of UOV

details not relevant to the design paradigm are neglected


Helper functions

In [1]:
# evaluate Multivariate map 
def MQeval(F,x,y):
    return [(x.transpose()*M*y)[0,0] for M in F]

def MQevalleft(F,x):
    return [ x.transpose()  *M for M in F]

In [2]:
# generate random invertible matrix 
def RandomInvertible(n):
    retM = Matrix(K,n,n)
    while not retM.is_invertible():
        retM = Matrix(K,[[K.random_element() for j in range(n)] for i in range(n)])
    return retM

# turn random matrix to upper diagonal
def SquareToUpper(M):
    n = M.ncols()
    retM = M
    for i in range(n):
        for j in range(i+1,n):
            retM[i,j] += retM[j,i]
            retM[j,i] = K(0) 
    return retM
            
# turn upper diagonal matrix to symmetric
def UpperToSymmetric(M):
    return M+M.transpose() 

UOV Key generation

In [3]:
# generates a UOV public-private key pair
# we neglect the 0s in the public/private key matrices when calculating key sizes, 
# and assume a triangular form is equivalent to a compressed representation 
def Keygen(q,n,m):
    Central_Map = [];
    for k in range(m):
        P  = Matrix(K,n,n)
        for i in range(n-m):
            for j in range(i,n):
                P[i,j] = K.random_element()
        Central_Map.append(P)
    S = RandomInvertible(n)
    Public_Key = [SquareToUpper(S.transpose()*M*S) for M in Central_Map]
    
    return Public_Key, S, Central_Map

Signing algorithm

In [4]:
def Sign(hash, S, Central_Map):
    Central_vinegar = [M.submatrix(0,0,n-m,n-m) for M in Central_Map]
    Central_oilvinegar = [M.submatrix(0,n-m,n-m,m) for M in Central_Map]
    coef_matrix = Matrix(K,m,m)
    while not coef_matrix.is_invertible():
        vinegar_vector=Matrix(K,n-m,1,[K.random_element() for j in range(n-m)])
        coef_matrix = Matrix(K,[(vinegar_vector.transpose() * Central_oilvinegar[i]).list()  for i in range(m)])
    coef_vector = Matrix(K,m,1,[(hash[i,0] - vinegar_vector.transpose() * Central_vinegar[i] * vinegar_vector)[0,0]  for i in range(m)])
    oil_vector = coef_matrix.inverse() * coef_vector
    signature = S.inverse() * vinegar_vector.stack(oil_vector)
    return signature

Verification algorithm

In [5]:
def Verify(hash, signature, Public_Key):
    hashprime = MQeval(Public_Key,signature,signature)
    return hash.list() == hashprime

Test for given parameters

In [6]:
# q=256
# n=112
# m=44

# q=16
# n=112
# m=44

q=4
n=28
m=11

# q=4
# n=84
# m=34

# q=4
# n=100
# m=40

# q=4
# n=56
# m=22

v=n-m
K = GF(q) 

In [7]:
Public_Key, S, Central_Map = Keygen(q,n,m)
hash = Matrix(K,m,1,[K.random_element() for j in range(m)])
print("Hash of message: ", hash.list())

Hash of message:  [z2, 1, 0, z2 + 1, z2 + 1, z2 + 1, 0, z2 + 1, 1, 0, 0]


In [8]:
signature = Sign(hash, S, Central_Map)
print("Signature: ", signature.list())  

Signature:  [1, 0, z2 + 1, 1, 0, z2 + 1, 0, z2, z2 + 1, z2, 0, 1, 0, z2, z2 + 1, 1, z2, z2 + 1, 1, z2, 0, z2, z2, 0, z2 + 1, z2 + 1, z2 + 1, 1]


In [9]:
verification = Verify(hash, signature, Public_Key)
print("Signature verification: ", verification)  

Signature verification:  True
